In [73]:
import scipy as scp
import scipy.sparse
import scipy.sparse.linalg
from scipy.linalg import expm

from scipy.sparse import csr_array

import matplotlib.pyplot as plt

import numpy as np

import functools # for functools.reduce

# Tools


In [74]:
def exp_mat(M) :
    '''
    Computes the matrix exponential exp(M)
    
    Parameters
    ----------
    M : csr_array
        matrix

    Return
    ------
    exp(M) : csr_array
    '''
    return scp.sparse.csr_array(expm(M.toarray()))

# Spin operators


In [24]:
sx = scipy.sparse.csr_array([[0.,  1.],[1., 0.]])
sy = scipy.sparse.csr_array([[0., -1.j],[1.j, 0.]])
sz = scipy.sparse.csr_array([[1., 0.],[0., -1.]])

I = scipy.sparse.identity(2)

sp = ((sx + 1.j * sy) / 2).real
sm = ((sx - 1.j * sy) / 2).real

$$
\hat{\sigma}^{z}_{j} = \underbrace{\hat{I}\otimes\hat{I}\dots\hat{I}}_{j-1\ \mathrm{terms}}\otimes\hat{\sigma}^{z}\otimes\underbrace{\hat{I}\otimes\hat{I}\dots\otimes\hat{I}}_{N-j\ \mathrm{terms}},
$$


In [49]:
def sz_j(j, N):
    """
    Compute the operator $Z_j$ acting on a N-spin system.
    
    Parameters
    ----------
    j : int
        The index of the spin to act on.
    N : int
        The number of spins in the system.

    Returns
    -------
    scipy.sparse.csr_matrix
        The operator $Z_j$.
    """
    return functools.reduce(scipy.sparse.kron, [I] * (j - 1) + [sz] + [I] * (N - j))


def sp_j(j, N):
    """
    Compute the operator $S^+_j$ acting on a N-spin system.
    
    Parameters
    ----------
    j : int
        The index of the spin to act on.
    N : int
        The number of spins in the system.

    Returns
    -------
    scipy.sparse.csr_matrix
        The operator $S^+_j$.
    """
    return functools.reduce(scipy.sparse.kron, [I] * (j - 1) + [sp] + [I] * (N - j))


def sm_j(j, N):
    """
    Compute the operator $S^-_j$ acting on a N-spin system.
    
    Parameters
    ----------
    j : int
        The index of the spin to act on.
    N : int
        The number of spins in the system.

    Returns
    -------
    scipy.sparse.csr_matrix
        The operator $S^-_j$.
    """
    return functools.reduce(scipy.sparse.kron, [I] * (j - 1) + [sm] + [I] * (N - j))


# Hamiltonian


In [46]:
def H_spin(N, J):
    """
    Compute the Hamiltonian of an N-spin system with nearest-neighbor interactions.

    Parameters
    ----------
    N : int
        The number of spins in the system.
    J : float
        The coupling constant.

    Returns
    -------
    scipy.sparse.csr_matrix
        The Hamiltonian of the system.
    """
    H = 0

    for i in range(1, N+1):
        if i == N:
            ip1 = 1
        else:
            ip1 = i + 1
        H += J * (sz_j(i, N) @ sz_j(ip1, N))
        
    return H

N = 2
J= 1

H = H_spin(N, J)
H.toarray()

array([[ 2.,  0.,  0.,  0.],
       [ 0., -2.,  0.,  0.],
       [ 0.,  0., -2.,  0.],
       [ 0.,  0.,  0.,  2.]])

# Lindbladian


Lindblad operators for the 2nd dissipator:


In [66]:
def get_Lks(N) :
    """
    Compute the L_k operators for a N-spin system (2nd dissipator).

    Parameters
    ----------
    N : int
        The number of spins in the system.

    Returns
    -------
    list of scipy.sparse.csr_matrix
        The L_k operators.

    Note
    ----
    The L_k operators are indexed from 1 to N: get_Lks(N)[k] corresponds to L_k,
    and get_Lks(N)[0] is a dummy value to have the same indexing as in the paper.
    """

    Lks = [None]   # in order to have the same indexing as in the paper: from 1 to N

    for k in range(1, N+1) :
        kp1 = (k + 1) % N
        kp2 = (k + 2) % N

        if kp1 == 0:
            kp1 = N
        if kp2 == 0:
            kp2 = N

        sp_k = sp_j(k, N)
        sm_kp1 = sm_j(kp1, N)
        sp_kp2 = sp_j(kp2, N)

        Lk = 1/2 * (sp_k @ sm_kp1 + 1j * sm_kp1 @ sp_kp2)
        Lks.append(Lk)

    return Lks

Lks = get_Lks(N)
Lks


[None,
 <4x4 sparse matrix of type '<class 'numpy.complex128'>'
 	with 2 stored elements (blocksize = 2x1) in Block Sparse Row format>,
 <4x4 sparse matrix of type '<class 'numpy.complex128'>'
 	with 1 stored elements in Compressed Sparse Row format>]

Dissipators:


In [67]:
def dissipator_1(rho, J, N) :
    """
    Compute the 1st dissipator of an N-spin system with nearest-neighbor interactions.

    Parameters
    ----------
    rho : scipy.sparse.csr_matrix
        The density matrix of the system.
    J : float
        The coupling constant.
    N : int
        The number of spins in the system.

    Returns
    -------
    scipy.sparse.csr_matrix
        The 1st dissipator of the system.
    """
    D = 0
    for k in range(N) :
        D += J * (sz_j(k, N) @ rho @ sz_j(k, N) - 0.5 * (sz_j(k, N) @ sz_j(k, N) @ rho + rho @ sz_j(k, N) @ sz_j(k, N)))
    return D

def dissipator_2(rho, J, N) :
    """
    Compute the 2nd dissipator of an N-spin system with nearest-neighbor interactions.

    Parameters
    ----------
    rho : scipy.sparse.csr_matrix
        The density matrix of the system.
    J : float
        The coupling constant.
    N : int
        The number of spins in the system.

    Returns
    -------
    scipy.sparse.csr_matrix
        The 2nd dissipator of the system.
    """
    D = 0
    for k in range(1, N+1) :
        D += J * (Lks[k] @ rho @ Lks[k].H - 0.5 * (Lks[k].H @ Lks[k] @ rho + rho @ Lks[k].H @ Lks[k]))
    return D

Lindbladian


In [69]:
def lindblad_non_unitary(rho, epsilon, gamma, J, N) :
    """
    Compute the non-unitary part of the Lindblad equation of an N-spin system with nearest-neighbor interactions.

    Parameters
    ----------
    rho : scipy.sparse.csr_matrix
        The density matrix of the system.
    epsilon : float
        The perturbation strength.
    gamma : float
        The relative strength.
    J : float
        The coupling constant.
    N : int
        The number of spins in the system.

    Returns
    -------
    scipy.sparse.csr_matrix
        The non-unitary part of the Lindbladian of the system.
    """
    L1 = epsilon * (gamma * dissipator_1(rho, J, N) + (1 - gamma) * dissipator_2(rho, J, N))
    return L1 


def lindblad(rho, H, epsilon, gamma, J, N) :
    """
    Compute the Lindblad equation of an N-spin system with nearest-neighbor interactions.

    Parameters
    ----------
    rho : scipy.sparse.csr_matrix
        The density matrix of the system.
    H : scipy.sparse.csr_matrix
        The Hamiltonian of the system.
    epsilon : float
        The perturbation strength.
    gamma : float
        The relative strength.
    J : float
        The coupling constant.
    N : int
        The number of spins in the system.

    Returns
    -------
    scipy.sparse.csr_matrix
        The Lindbladian of the system.
    """
    L0 = - 1j * (H @ rho - rho @ H)
    L1 = lindblad_non_unitary(rho, epsilon, gamma, J, N)
    return L0 + L1 

# Conserved quantities


In [71]:
def Ob(N) :
    """
    Boost operator

    Parameters
    ----------
    N : int
        The number of spins in the system.

    Returns
    -------
    scipy.sparse.csr_matrix
        The boost operator.
    """
    O = 0

    for i in range(1, N+1):
        if i == N:
            ip1 = 1
        else:
            ip1 = i + 1
        O += -1j * i * (sz_j(i, N) @ sz_j(ip1, N))
        
    return O




In [70]:
N = 2
J = 1

C2 = H_spin(N, J)



Cs = [
    
    
    
]

# GGE


In [ ]:
def GGE(lagr, conserved_quantities) :
    """
    Compute the Generalized Gibbs Ensemble (GGE) density matrix, given the Lagrange multipliers and the conserved quantities.

    Parameters
    ----------
    lagr : list of float
        The Lagrange multipliers.

    conserved_quantities : list of scipy.sparse.csr_matrix
        The conserved quantities.

    Returns
    -------
    scipy.sparse.csr_matrix
        The GGE density matrix.
    """    
    exponent = 0
    for lambda_i, C_i in zip(lagr, conserved_quantities) :
        exponent -= lambda_i * C_i

    U = exp_mat(exponent)
    rho = U / scp.sparse.csr_matrix.trace(U)
    return rho

In [ ]:
# TODO
# implement heat current JH
# implement the conserved quantities
# implement function: (X,rho_GGE) -> Tr[X @ rho_GGE]
# build correlation matrix of the Lagrange multipliers ('chi')
# implement function that computes Fi
# numerical scheme for the lambdas, given Fi'
# what is a Néel state ?
# solve the system of equations for the lambdas: time-dependent GGE !
# compute some observables (like in the paper) and compare with the exact solution
# link everything with variational methods
 